<a href="https://colab.research.google.com/github/avocadopelvis/mental-load/blob/main/mental_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
# #LOAD DATA
# df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
# df = df.drop(df.columns[0], axis = 1)
# df

In [4]:
# use gpu if available 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [30]:
df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
df = df.drop(df.columns[0], axis = 1)

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state = 42)

In [31]:
class SignalDataset(Dataset):
    def __init__(self, df):
      # load dataset
      # df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
      # drop first column
      # df = df.drop(df.columns[0], axis = 1)
      # get x and convert to tensor
      self.x = torch.tensor(df.iloc[:, 0:3840].values)
      # get y and convert to tensor
      self.y = torch.tensor(df["label"].values)
      # get number of samples
      self.n_samples = df.shape[0]

    def __len__(self):
      return self.n_samples

    def __getitem__(self, idx):
      return self.x[idx], self.y[idx]

In [32]:
train_dataset = SignalDataset(train)
test_dataset = SignalDataset(test)
# first = data[0]
# X, y = first
# print(X, y)

# Preparing data for training with DataLoaders

In [34]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True) #,num_workers = 2
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [18]:
# dataiter = iter(train_dataloader)
# data = dataiter.next()
# features, labels = data
# print(features, labels)
# len(features[0])

3840

In [10]:
# from torch import nn

# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(28*28, 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, 10),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

In [19]:
from torch import nn

# hyperparameters
input_size = 3840
hidden_size = 100
num_classes = 3
num_epochs = 15
batch_size = 10
learning_rate = 0.001

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out

model = NeuralNetwork(input_size, hidden_size, num_classes)

In [11]:
# model = NeuralNetwork().to(device)
# print(model)

In [20]:
# LOSS & OPTIMIZER
criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [35]:
n_total_steps = len(train_dataloader)

for epoch in range(num_epochs):
  for i, data in enumerate(train_dataloader, 0):
    # get the inputs and labels
    inputs, labels = data

    # forward
    outputs = model(inputs.float())
    loss = criterion(outputs, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

# TEST
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)

    # value, index (We don't need value)
    _, predictions = torch.max(outputs, 1)
    n_samples == labels.shape[0]
    n_correct == (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

epoch 1 / 15, step 100 / 189, loss = 0.0009
epoch 2 / 15, step 100 / 189, loss = 0.0013
epoch 3 / 15, step 100 / 189, loss = 0.0023
epoch 4 / 15, step 100 / 189, loss = 0.0016
epoch 5 / 15, step 100 / 189, loss = 0.0006
epoch 6 / 15, step 100 / 189, loss = 0.0013
epoch 7 / 15, step 100 / 189, loss = 0.0003
epoch 8 / 15, step 100 / 189, loss = 0.0004
epoch 9 / 15, step 100 / 189, loss = 0.0003
epoch 10 / 15, step 100 / 189, loss = 0.0004
epoch 11 / 15, step 100 / 189, loss = 0.0004
epoch 12 / 15, step 100 / 189, loss = 0.0003
epoch 13 / 15, step 100 / 189, loss = 0.0002
epoch 14 / 15, step 100 / 189, loss = 0.0002
epoch 15 / 15, step 100 / 189, loss = 0.0001


RuntimeError: ignored

In [ ]:
# TEST
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)

    # value, index (We don't need value)
    _, predictions = torch.max(outputs, 1)
    n_samples == labels.shape[0]
    n_correct == (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

In [13]:
# for epoch in range(2):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(dataloader, 0):
#         # get the inputs and labels
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs)
          # calculate loss
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0

# print('Finished Training')

In [14]:
# class SignalDataset(Dataset):
#     def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
#         self.labels = y
#         self.samples = X
#         self.transform = transform
#         self.target_transform = target_transform

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         path = os.path.join(self.samples.iloc[idx, 0], self.labels.iloc[idx, 0])
#         sample = self.samples.iloc[idx, 0]  #FIGURE OUT HOW TO READ SAMPLE
#         label = self.labels.iloc[idx, 1]
#         if self.transform:
#             sample = self.transform(sample)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return sample, label